In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
df= pd.read_csv("/content/drive/MyDrive/Dataset-SA.csv")
df = df.dropna()
df=df.drop("Rate",axis =1)
df= df.drop("product_name",axis =1)
df = df.drop("product_price",axis =1)
df

,Review,Summary,Sentiment
0,super!,great cooler excellent air flow and for this p...,positive
1,awesome,best budget 2 fit cooler nice cooling,positive
2,fair,the quality is good but the power of air is de...,positive
3,useless product,very bad product its a only a fan,negative
4,fair,ok ok product,neutral
...,...,...,...
205047,must buy!,good product,positive
205048,super!,nice,positive
205049,nice,very nice and fast delivery,positive
205050,just wow!,awesome product,positive


In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.3 MB/s eta 0:00:00


In [ ]:
sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
df['Sentiment'] = df['Sentiment'].map(sentiment_map)
df['Sentiment']
# df['Sentiment']

0         2
1         2
2         2
3         0
4         1
         ..
205047    2
205048    2
205049    2
205050    2
205051    1
Name: Sentiment, Length: 180370, dtype: int64

In [ ]:

from sklearn.model_selection import train_test_split

# Split the dataset into input features (X) and output variable (y)
X = df.iloc[:, :-1]
y = df.iloc[:, 2:3]


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Split the training data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

In [ ]:
from transformers import DistilBertTokenizerFast

# Instantiate DistilBERT tokenizer...we use the Fast version to optimize runtime
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
import tensorflow as tf
MAX_LENGTH = 128


# Define function to encode text data in batches
def batch_encode(tokenizer, texts, batch_size=256, max_length=MAX_LENGTH):
    input_ids = []
    attention_mask = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer.batch_encode_plus(batch,
                                             max_length=max_length,
                                             padding='longest', #implements dynamic padding
                                             truncation=True,
                                             return_attention_mask=True,
                                             return_token_type_ids=False
                                             )
        input_ids.extend(inputs['input_ids'])
        attention_mask.extend(inputs['attention_mask'])
        
    input_ids = tf.keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_length, padding='post', truncating='post')
    attention_mask = tf.keras.preprocessing.sequence.pad_sequences(attention_mask, maxlen=max_length, padding='post', truncating='post')
    
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_mask)
    
    


In [ ]:
 X_train.values.tolist()

[['wonderful', 'nice working'],
 ['not specified', 'awesome product in lowest price'],
 ['nice', 'refill not satisfied'],
 ['not specified', 'very good'],
 ['nice product',
  'i bought it for 6000 in bbd sale packaging was ok ok bt product arrived without any damage looks are awesome quality is nice installing is also easy u can do it ur self over all satisfactory'],
 ['brilliant', 'very good product'],
 ['delightful', 'good'],
 ['awesome', 'more than enough'],
 ['excellent', 'nice'],
 ['nice', 'excellent working'],
 ['worth the money', 'nyc product'],
 ['super!',
  'sorry to say but flipkart complaint system is not working at all i have raised an issue against this devise almost 5 days back and receive the deaf ear treatment from the flipkart staffthis is outrageous and really annoying no response received from the staff'],
 ['useless product', 'low suction power'],
 ['excellent', 'very good product must buy'],
 ['super!', 'super'],
 ['great product',
  'its really amazing and crazy a

In [ ]:
# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train.values.tolist())

# Encode X_valid
X_valid_ids, X_valid_attention = batch_encode(tokenizer, X_valid.values.tolist())

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test.values.tolist())


In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)
                          
# The bare, pre-trained DistilBERT transformer model outputting raw hidden-states 
# and without any specific head on top.
distilBERT = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)

# Make DistilBERT layers untrainable
for layer in distilBERT.layers:
    layer.trainable = False

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
# !pip install focal_loss

In [ ]:
# Set parameters:
params = {'MAX_LENGTH': 128,
          'EPOCHS': 6,
          'LEARNING_RATE': 5e-5,
          'FT_EPOCHS': 6,
          'OPTIMIZER': 'adam',
          'FT_LEARNING_RATE': 2e-5,
          'BATCH_SIZE': 64,
          'NUM_STEPS': len(X_train) // 64,
          'DISTILBERT_DROPOUT': 0.2,
          'DISTILBERT_ATT_DROPOUT': 0.2,
          'LAYER_DROPOUT': 0.2,
          'KERNEL_INITIALIZER': 'GlorotNormal',
          'BIAS_INITIALIZER': 'zeros',
          'POS_PROBA_THRESHOLD': 0.5,          
          'ADDED_LAYERS': 'Dense 256, Dense 32, Dropout 0.2',
          'LR_SCHEDULE': '5e-5 for 6 epochs, Fine-tune w/ adam for 6 epochs @2e-5',
          'FREEZING': 'All DistilBERT layers frozen for 6 epochs, then unfrozen for 6',
          'CALLBACKS': '[early_stopping monitoring val_loss w/ patience=0]',
          'RANDOM_STATE':42
          }


In [ ]:
!pip install focal_loss

In [ ]:
MAX_LENGTH = 128
LAYER_DROPOUT = 0.2
LEARNING_RATE = 5e-5
RANDOM_STATE = 42

# from focal_loss import BinaryFocalLoss

def build_model(transformer, max_length=MAX_LENGTH):
    # Define weight initializer with a random seed to ensure reproducibility
    weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE) 
    
    # Define input layers
    input_ids_layer = tf.keras.layers.Input(shape=(max_length,), 
                                            name='input_ids', 
                                            dtype='int32')
    input_attention_layer = tf.keras.layers.Input(shape=(max_length,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    

    last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]


    cls_token = last_hidden_state[:, 0, :]

    D1 = tf.keras.layers.Dropout(params['LAYER_DROPOUT'],
                                 seed=params['RANDOM_STATE']
                                )(cls_token)
    
    X = tf.keras.layers.Dense(256,
                              activation='relu',
                              kernel_initializer=weight_initializer,
                              bias_initializer='zeros'
                              )(D1)
    
    D2 = tf.keras.layers.Dropout(params['LAYER_DROPOUT'],
                                 seed=params['RANDOM_STATE']
                                )(X)
    
    X = tf.keras.layers.Dense(32,
                              activation='relu',
                              kernel_initializer=weight_initializer,
                              bias_initializer='zeros'
                              )(D2)
    
    D3 = tf.keras.layers.Dropout(params['LAYER_DROPOUT'],
                                 seed=params['RANDOM_STATE']
                                )(X)
    
    # Define a single node that makes up the output layer (for binary classification)
    output = tf.keras.layers.Dense(3, 
                                   activation='softmax',
                                   kernel_initializer=weight_initializer,  
                                   kernel_constraint=None,
                                   bias_initializer='zeros'
                                   )(cls_token)
    
    # Define the model
    model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
    
    # Compile the model
    model.compile(tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), 
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),#=BinaryFocalLoss(gamma =1)
                  metrics=['accuracy'])
    
    return model

In [ ]:
model = build_model(distilBERT)


In [ ]:
import numpy as np
EPOCHS = 10
BATCH_SIZE = 64
NUM_STEPS = len(X_train) // BATCH_SIZE
NUM_STEPS


1380

In [ ]:
import numpy
train_history1 = model.fit(
    x = [X_train_ids, X_train_attention],
    y = y_train.to_numpy(),
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    steps_per_epoch = NUM_STEPS,
    validation_data = ([X_valid_ids, X_valid_attention],y_valid.to_numpy())
)

Epoch 1/10
1380/1380 [==============================] - 551s 393ms/step - loss: 0.6087 - accuracy: 0.7752 - val_loss: 0.4808 - val_accuracy: 0.8189
Epoch 2/10
1380/1380 [==============================] - 591s 428ms/step - loss: 0.4593 - accuracy: 0.8298 - val_loss: 0.4050 - val_accuracy: 0.8366
Epoch 3/10
1380/1380 [==============================] - 547s 396ms/step - loss: 0.4024 - accuracy: 0.8500 - val_loss: 0.3545 - val_accuracy: 0.8657
Epoch 4/10
1380/1380 [==============================] - 591s 428ms/step - loss: 0.3640 - accuracy: 0.8707 - val_loss: 0.3222 - val_accuracy: 0.8943
Epoch 5/10
1380/1380 [==============================] - 546s 396ms/step - loss: 0.3397 - accuracy: 0.8846 - val_loss: 0.3025 - val_accuracy: 0.9037
Epoch 6/10
1380/1380 [==============================] - 591s 428ms/step - loss: 0.3230 - accuracy: 0.8927 - val_loss: 0.2887 - val_accuracy: 0.9091
Epoch 7/10
1380/1380 [==============================] - 591s 428ms/step - loss: 0.3102 - accuracy: 0.8983 - val_